# Quantum Substring Comparison
#

In [8]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from math import log2, floor
import numpy as np
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_bloch_vector
import matplotlib.pyplot as plt  

## Introduction

In recent years has gained ground the emerging field of quantum computing,giving the possibility to implement algorithms with superior asymptotic performance to those achievable using classical computers.   
The aim of the project is to provide an implementation of a quantum algorithm [1] :https://arxiv.org/abs/2308.11758, which solve a classical problem related to text processing, and compare the performance with a classical implemention of the same algorithm.
Given two input strings, x and y, both of length n, and a value d ⩽ n, we want to verify the following conditions:  
the existence of a common prefix of length d, the presence of a common substring of length d beginning at position j  and, the presence  of any common substring of length d beginning in both strings at the same position.
Formally, given tow input strings x and y,both of length n, whose character are drown from an alphabet S, and a parameter d,with 0 < d ⩽ n, we want to solve the following problems:
- the Fixed Prefix Matching (FPM) problem, which tests for the existence of  a common prefix of length d between x and y, checks whether:  
   <pre>                                  x[0,1...d-1]= y[0,1...d-1]</pre>                                        
- the Fixed Factor Matching (FFM) problem, which test, for a given parameter j, with 0 ⩽ j < n−d , the existence of a common substring(of length d) between x and y, starting at index j .
   <pre>                                  x[j,j+1...j+d-1]= y[j,j+1...j+d-1]            </pre>
- the Shared Fixed Substring Checking (SFSC) problem, which tests for  the existence of a common substring between x and y, of length d,     starting at index j in both strings. It tests whether:
   <pre>                        ∃ j : 0 ⩽ j < n−d and x[j,j+1...j + d − 1] = y[j,j+1...j + d − 1]     <pre>                                             

## A gentle introduction to quantum Computing

Qbit is the basic unit of quantum computation and can be understood as the quantum equivalent of the traditional bit used in
classical computers.
It is a two-state quantum mechanical system, one of the simplest quantum systems displaying the peculiarity of quantum mechanics, 
examples include the spin of the electron or the polarization of a single photon.  
In quantum mechanics a general quantum state can be represented by a vector defined on Hilbert space  $\mathcal{H}$, in particular a qbit, that is a very simple quantum mechanical system, can be represented as linear combination of two basis states |0$\rangle$ and |1$\rangle$:
$$ |\psi\rangle = \alpha|0\rangle + \beta|1\rangle\ \alpha,\beta \in \mathbb{C}\ $$
where |$\alpha$|$^2$ and |$\beta$|$^2$ represent the probability of outcome  |0$\rangle$ and  |1$\rangle$ respectively. Because the absolute squares of the amplitudes equate to probabilities, it follows that $\alpha$ and $\beta$ must be constrained, according to the second axiom of probability theory, by the equation:
$$ |\alpha|^2 + |\beta|^2=1$$
This expression represent quantum superposition, a key concept that distinguishes quantum mechanical system from their classical counterpart.  
  
Multiple qbits taken together are referred to as a quantum registers.A quantum register |$\psi\rangle$=|q1,..qn$\rangle$ is the tensor product of constituent qbits:
$$ |\psi\rangle= \otimes_{i=1}^n q_i $$ 

### Quantum Operators

A qbit can be represented in the Bloch sphere.However, a qbit has four parameter but a sphere allows 
only three dimensional representation, so Bloch manipulated the definition to obtain this equation:
$$ \ket{\psi} = \cos \frac{\theta}{2} \ket{0} + e^{i \phi} \sin \frac{\theta}{2} \ket{1} $$

A quantum operation maps a point u on the unit sphere to some other point q on the unit sphere, 
so an operation can be represented by a matrix.
Moreover, we restrict these matrices to be linear and invertible, such matricies are called unitary.
- linearity
$$ A(\alpha \ket{\psi_1} + \beta \ket{\psi_2}) = \alpha A(\ket{\psi_1}) + \beta A(\ket{\psi_2})\
\forall \alpha, \beta \in \mathbb{C},\ \forall \ket{\psi_1}, \ket{\psi_2} \in \mathcal{H} $$
- invertibility
$$ U^\dagger U = I\ \ \
 UU^\dagger = I $$
                  where $U^\dagger$ is the complex conjugate of $U$ and $I$ is the identity matrix.   
The following are the elementary quantum operators we will use :
- NOT gate, formally it maps |0$\rangle$ to |1$\rangle$ and vice versa
$$
NOT = \begin{bmatrix} 0 & 1\\ 1 & 0\end{bmatrix}
$$ 

- CNOT gate, formally it maps |q$_0$,q$_1$ $\rangle$ to |q$_0$,q$_0$ $\oplus$q$_1$ $\rangle$
$$
CNOT = \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{bmatrix}
$$

- SWAP gate, formally it maps |q$_0$,q$_1$ $\rangle$ to |q$_1$,q$_0$ $\rangle$, it could be obtained applying three times CNOT
$$
SWAP = \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \end{bmatrix}
$$

- the FREDKIN gate, that consist in swap gate controlled by a single qbit
$$
FREDKIN = \begin{bmatrix} 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 1 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0  \\ 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0  \\ 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\end{bmatrix}
$$
- Toffoli gate, it  formally maps |q$_0$,q$_1$, q$_2$ $\rangle$ to |q$_0$,q$_1$, q$_0$ $\oplus$q$_1$ $\oplus$q$_2$ $\rangle$
$$
CX3 = \begin{bmatrix} 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 1 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0  \\ 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1  \\ 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0\end{bmatrix}
$$

- Multiple CNOT, formally it maps |q$_0$,q$_1$,q$_2$...q$_n$ $\rangle$ to |q$_0$,q$_1$,q$_2$... ,q$_0$ $\oplus$ q$_1$ $\oplus$ q$_2$ $\oplus$ q$_n$ $\rangle$

### Quantum Boolean Oracles

Given a function f: {0}$^n$ -> {1}$^n$,any quantum operator that maps a register containing the value of a given input x ∈ {0, 1}$^n$ into a register whose value depends on f (x) is called a quantum  boolean oracle.Formally,
$$
U_f|x,0\rangle=|x,f(x)\rangle
$$

### Quantum Complexity

 A quantum algorithm can be shaped using the quantum circuit computational model.It represent quantum algorithms as sequences
of quantum gates that manipulate qubits.This approach delves into the specifics
of how quantum operations are executed and provides a structured methodology
for designing and optimizing quantum algorithms.We assume the
circuit as being divided into a sequence of discrete time-steps, where the appli-
cation of a single gate requires a single time-step. The depth of the circuit is the
total number of required time-steps and it is the time complexity measure

## Algorithm 

FSM(d, x, y, D$^{-1}$ ):
1. λ0 = M(x, y)  
2. if( $\bar{d}$[0] = 1) then D$^{0}$ = (D$^{-1}$ ∧ λ$^{0}$ ) ≫ 2$^{0}$  
3. for i = 1 to log(d) do  
4.  λ$^{i}$ = EXT$_{i}$(λ$_{i-1}$)
5.  if ($\bar{d}$[i] = 1) then D$^i$ = (D$^{i-1}$ ∧ λ$^{i}$ ) ≫ 2$^{i}$  
6.  else D$^{i}$ = D$^{i-1}$
7.  end for 
8. r=V$_{j=d}^{n}$D$^{log(d)}$[j$\cdot$ mod(n)]  
9. return r